### data

In [303]:
import pandas as pd
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt
import gurobipy as gp
from gurobipy import GRB

In [304]:
alpha = pd.read_csv("result_alpha.csv").to_numpy().flatten()
beta_plus = pd.read_csv("result_beta_plus.csv").to_numpy()
beta_minus = pd.read_csv("result_beta_minus.csv").to_numpy()

R_flat = pd.read_csv("result_R.csv")

I_len = R_flat["generator"].nunique()
T_len = R_flat["time"].nunique() 
S_len = R_flat["scenario"].nunique()

R = R_flat.pivot_table(
    values="value",
    index="generator",
    columns=["time", "scenario"]
).to_numpy()
R = R.reshape(I_len, T_len, S_len)

### settlement model

In [305]:
I, T, S = R.shape
set = gp.Model("Settlement")
set.Params.OutputFlag = 0
set.setParam('TimeLimit', 3600)
set.setParam('MIPGap', 0.01)
set.setParam('Presolve', 0)

x = set.addVars(I, T, vtype=GRB.CONTINUOUS, lb=0, name="x")
y_plus = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="y_plus")
y_minus = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="y_minus")
M = max(sum(R[i, t, s] for i in range(I)) for t in range(T) for s in range(S))
z = set.addVars(I, T, S, vtype=GRB.BINARY, name="z")
d = set.addVars(I, I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="d")

set.update()

In [306]:
# x 값들 간 최대 차이를 최소화
obj = gp.quicksum((x[i, t] - alpha[t] / I)**2 for i in range(I) for t in range(T))
set.setObjective(obj, GRB.MINIMIZE)

In [307]:
# 기존 제약식

# x로 y+, y- 결정 지어주기
for i in range(I):
    for t in range(T):
        for s in range(S):
            set.addConstr(R[i, t, s] - x[i, t] == y_plus[i, t, s] - y_minus[i, t, s])

# real-time market 참여 제약
for i in range(I):
    for t in range(T):
        for s in range(S):
            set.addConstr(R[i,t,s] - y_plus[i, t, s] >= 0)

# y+, y- 둘 중 하나는 0이어야함
for i in range(I):
    for t in range(T):
        for s in range(S):
            set.addConstr(y_plus[i, t, s] <= M * z[i, t, s])
            set.addConstr(y_minus[i, t, s] <= M * (1 - z[i, t, s]))

# alpha는 모든 x의 합이다
for t in range(T):
    set.addConstr(
        gp.quicksum(x[i, t] for i in range(I)) == alpha[t]
    )


In [308]:
# 초과량이 있는 발전기만 다른 발전기에게 전력을 줄 수 있음
for i in range(I):
    for t in range(T):
        for s in range(S):
            set.addConstr(
                gp.quicksum(d[i, j, t, s] for j in range(I) if j != i) <= M * z[i, t, s]
            )

# 자기 자신과의 거래 방지
for i in range(I):
    for t in range(T):
        for s in range(S):
            set.addConstr(d[i, i, t, s] == 0)

# β+, β- 매칭
for t in range(T):
    for s in range(S):
        # 실제 남는 양(초과량 - 주는 양)의 합이 beta+와 같아야 함
        set.addConstr(
            gp.quicksum(
                y_plus[i, t, s] - gp.quicksum(d[i, j, t, s] for j in range(I) if j != i)
                for i in range(I)
            )
            == beta_plus[t, s]
        )

        # 실제 부족한 양(부족량 - 받는 양)의 합이 beta-와 같아야 함
        set.addConstr(
            gp.quicksum(
                y_minus[i, t, s]
                - gp.quicksum(d[j, i, t, s] for j in range(I) if j != i)
                for i in range(I)
            )
            == beta_minus[t, s]
        )

# 전력 이동 제한
for i in range(I):
    for t in range(T):
        for s in range(S):
            # 발전기가 주는 총량은 자신의 초과량을 넘을 수 없음
            set.addConstr(
                gp.quicksum(d[i, j, t, s] for j in range(I) if j != i)
                <= y_plus[i, t, s]
            )

            # 발전기가 받는 총량은 자신의 부족량을 넘을 수 없음
            set.addConstr(
                gp.quicksum(d[j, i, t, s] for j in range(I) if j != i)
                <= y_minus[i, t, s]
            )

In [309]:
set.optimize()

if set.status == GRB.OPTIMAL:
    x_vals = np.array([[x[i, t].x for t in range(T)] for i in range(I)])
    y_plus_vals = np.array([[[y_plus[i, t, s].x for s in range(S)] for t in range(T)] for i in range(I)])
    y_minus_vals = np.array([[[y_minus[i, t, s].x for s in range(S)] for t in range(T)] for i in range(I)])
    d_vals = np.array([[[[d[i,j,t,s].x for s in range(S)] for t in range(T)] for j in range(I)] for i in range(I)])
    print("Optimal solution found!")
    print(set.objVal)
else:
    print("\nNo optimal solution found.")

Optimal solution found!
2.6147972675971687e-12


In [310]:
print(f"Status: {set.status}")
if set.status == GRB.OPTIMAL:
    print("모델이 Optimal입니다")
elif set.status == GRB.UNBOUNDED:
    print("모델이 Unbounded입니다")
    # Unbounded ray 확인
    set.computeIIS()
    set.write("model.ilp")
elif set.status == GRB.INFEASIBLE:
    print("모델이 Infeasible입니다")
    # IIS 계산
    set.computeIIS()
    set.write("model.ilp")
elif set.status == GRB.INF_OR_UNBD:
    print("모델이 Infeasible 또는 Unbounded입니다")
    # Presolve를 끄고 다시 시도
    set.setParam('Presolve', 0)
    set.optimize()
elif set.status == GRB.TIME_LIMIT:
    print("시간 제한에 도달했습니다")
    if set.SolCount > 0:
        print(f"현재까지의 최선해: {set.objVal}")

Status: 2
모델이 Optimal입니다


### 결과 분석

In [311]:
for t in range(12, 14):
    x_sum = sum(x_vals[i, t] for i in range(I))

    for s in range(2,3):
        y_plus_sum = sum(y_plus_vals[i, t, s] for i in range(I))
        y_minus_sum = sum(y_minus_vals[i, t, s] for i in range(I))

        y_plus_settled = [
            y_plus_vals[i, t, s] - sum(d_vals[i, j, t, s] for j in range(I) if j != i)
            for i in range(I)
        ]
        y_minus_settled = [
            y_minus_vals[i, t, s] - sum(d_vals[j, i, t, s] for j in range(I) if j != i)
            for i in range(I)
        ]

        y_plus_settled_sum = sum(y_plus_settled)
        y_minus_settled_sum = sum(y_minus_settled)

        print(f"[t={t} s={s}]")

        print(f"alpha: {alpha[t]:.2f}")

        print(f"x 합계: {x_sum:.2f}     (", end="")
        for i in range(I):
            print(f"[{i}] {x_vals[i,t]:.2f}", end=" ")
        print(")")

        print(f"y+ 합계:     {y_plus_sum:.2f} (", end="")
        for i in range(I):
            print(f"[{i}] {y_plus_vals[i,t,s]:.2f}", end=" ")
        print(")")

        print(f"y- 합계:     {y_minus_sum:.2f} (", end="")
        for i in range(I):
            print(f"[{i}] {y_minus_vals[i,t,s]:.2f}", end=" ")
        print(")")

        print(f"y+ set 합계: {y_plus_settled_sum:.2f} (", end="")
        for i in range(I):
            print(f"[{i}] {y_plus_settled[i]:.2f}", end=" ")
        print(")")

        print(f"y- set 합계: {y_minus_settled_sum:.2f} (", end="")
        for i in range(I):
            print(f"[{i}] {y_minus_settled[i]:.2f}", end=" ")
        print(")")

        print(f"beta+:      {beta_plus[t,s]:.2f}")
        print(f"beta-:      {beta_minus[t,s]:.2f}")

        print("거래 내역:")
        for i in range(I):
            for j in range(I):
                if d_vals[i,j,t,s] > 1e-6:  # 수치 오차 고려하여 매우 작은 값 제외
                    print(f"발전기 {i}가 발전기 {j}에게 {d_vals[i,j,t,s]:.2f} 전력을 줌")
        print()

[t=12 s=2]
alpha: 58.60
x 합계: 58.60     ([0] 3.08 [1] 3.08 [2] 3.08 [3] 3.08 [4] 3.08 [5] 3.08 [6] 3.08 [7] 3.08 [8] 3.08 [9] 3.08 [10] 3.08 [11] 3.08 [12] 3.08 [13] 3.08 [14] 3.08 [15] 3.08 [16] 3.08 [17] 3.08 [18] 3.08 )
y+ 합계:     37.89 ([0] 7.07 [1] 0.00 [2] 1.10 [3] 5.50 [4] 0.85 [5] 5.07 [6] 7.90 [7] 0.93 [8] 0.00 [9] 1.39 [10] 5.10 [11] 0.00 [12] 0.00 [13] 1.29 [14] 0.00 [15] 0.00 [16] 0.00 [17] 0.00 [18] 1.69 )
y- 합계:     18.00 ([0] 0.00 [1] 2.33 [2] 0.00 [3] 0.00 [4] 0.00 [5] 0.00 [6] 0.00 [7] 0.00 [8] 3.08 [9] 0.00 [10] 0.00 [11] 0.49 [12] 3.08 [13] 0.00 [14] 0.32 [15] 3.08 [16] 3.08 [17] 2.52 [18] 0.00 )
y+ set 합계: 19.89 ([0] 4.75 [1] 0.00 [2] 1.10 [3] 0.00 [4] 0.85 [5] 1.99 [6] 1.48 [7] 0.93 [8] 0.00 [9] 1.39 [10] 4.43 [11] 0.00 [12] 0.00 [13] 1.29 [14] 0.00 [15] 0.00 [16] 0.00 [17] 0.00 [18] 1.69 )
y- set 합계: 0.00 ([0] 0.00 [1] 0.00 [2] 0.00 [3] 0.00 [4] 0.00 [5] 0.00 [6] 0.00 [7] 0.00 [8] 0.00 [9] 0.00 [10] 0.00 [11] 0.00 [12] 0.00 [13] 0.00 [14] 0.00 [15] 0.00 [16] 0.00 

In [312]:
for t in range(T):
    for i in range(I):
        print(f"t={t} i={i}")
        print(f"x={x_vals[i,t]:.4f}")
        print(f"y+=", end=" ")
        for s in range(S):
            print(f"{y_plus_vals[i,t,s]:.4f}", end=" ")
        print("\ny-=", end=" ")
        for s in range(S):
            print(f"{y_minus_vals[i,t,s]:.4f}", end=" ")
        print("\nR=", end=" ")
        for s in range(S):
            print(f"{R[i,t,s]:.4f}", end=" ")
        print("\n")

t=0 i=0
x=0.0000
y+= 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 
y-= 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 
R= 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 

t=0 i=1
x=0.0000
y+= 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 
y-= 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 
R= 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 

t=0 i=2
x=0.0000
y+= 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.0000 0.000